In [ ]:
!pip install readlif

In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import matplotlib
matplotlib.use('Qt5Agg')
import glob
from readlif.reader import LifFile
import tifffile as tiff
import numpy as np
import cv2

In [ ]:
# base_dir includes subject folders, each contains the image files in .lif format
base_dir = "Volumes/Extreme SSD/RNAscope/CIRL1/Plexus injury patients/NF200"

In [1]:
def project_image(im_array, method = 'max'):
    """
    process the image so that it gives you back the max_projection, mean projection or EDF
    :param im_array: image as numpy array
    :param method: projection method. Can be either 'max' for maximum projection intensity,
                   'mean' for average intensity projection or EDF for Extended depth of focus
    :return: projected image
    """
    final_image = np.zeros((30,30,4))
    if method == 'max':
        final_image = np.max(im_array,axis = 0)
    elif method == 'mean':
        final_image = np.average(im_array, axis=0).astype(im_array.dtype)
    elif method == 'edf':
        # Initialize an empty array to hold the focus measure for each Z-plane
        focus_measure_stack = np.zeros(im_array.shape[:3])

        for z in range(im_array.shape[0]):
            focus_measure_stack[z] = cv2.Laplacian(im_array[z], cv2.CV_64F).var()

        best_focus_indices = np.argmax(focus_measure_stack, axis=0)

        edf_image = np.zeros(im_array.shape[1:], dtype=im_array.dtype)
        for i in range(im_array.shape[1]):
            for j in range(im_array.shape[2]):
                edf_image[i, j] = im_array[best_focus_indices[i, j], i, j]

        final_image = edf_image

    return final_image

In [ ]:
def extract_tif(filepath, method = 'max', low_memory = True):
    """
    Extract one representative .lif image out of the z-stacked .lif images
    :param filepath: path to the .lif file to be opened
    :param method: method of channel stacking. Can be either 'max' for maximum projection intensity,
                   'mean' for average intensity projection or EDF for Extended depth of focus
    :param low_memory: if you have low RAM in your PC, you can choose this option, the function will read
                       the z-stacks 5 at a time. This only works with method == max
    :return: final image as numpy array
    """
    parent_dir = os.path.dirname(filepath)
    new_parent_dir = parent_dir+'_tif'
    filename = filepath.replace('\\','/').split('/')[-1]
    file = LifFile(filepath)

    print(f"\033[1m\033[94m--> Extracting image {filename}\033[0m")

    lif_image = file.get_image(file.num_images-1) # read lif image
    z_count = lif_image.info['dims'].z # number of z-stacked layers
    ch_count = lif_image.channels # number of channels

    # Create 4 lists of z-stacked images. This is done in order to apply the maximum intensity projection
    # on each channel separately ... impartant to save memoory on these severly hindered machines!!
    channel_list = []
    colors = ['92m','94m','91m','96m'] # just to make the output colorful
    ch_names = ['NF200','DAPI','P2RX3','CIRL1/3']
    if low_memory and method =='max': # This helps if you have low memory (8 GB of RAM)
        channel_list = []
        for ch in range(ch_count):
            z_range_list = np.arange(z_count)
            chunk_size = 10
            z_ranges = [z_range_list[i:i+chunk_size] for i in range(0,len(z_range_list),chunk_size)]
            proj_image_list=[]
            for z_range in z_ranges:
                z_stacked_single_channel_loc = np.array([np.array(lif_image.get_frame(z=z, c=ch)) for z in z_range])
                proj_image_loc = project_image(z_stacked_single_channel_loc, method=method)
                proj_image_list.append(proj_image_loc)
            proj_image = project_image(np.array(proj_image_list),method = method)
            channel_list.append(proj_image)
            print(f'\033[{colors[ch]}processed {ch_names[ch]}\033[0m')
    else:
        for ch in range(ch_count):
            z_stacked_single_channel = np.array([np.array(lif_image.get_frame(z=z, c=ch)) for z in range(z_count)])
            proj_image = project_image(z_stacked_single_channel, method=method)
            channel_list.append(proj_image)
            print(f'\033[{colors[ch]}processed {ch_names[ch]} \033[0m')

    print(f'Done processing {filename} <--\n')
    final_images = np.array(channel_list)

    if not os.path.exists(new_parent_dir):
        os.mkdir(new_parent_dir)

    for final_image, stain_name in zip(final_images, ['NF200','DAPI','P2RX3','CIRL13']):
        tiff.imwrite(f"{new_parent_dir}/{filename.replace('.lif',f'_stain-{stain_name}.tiff')}", final_image)

    return final_image

all_files = glob.glob(f'{base_dir}/*/*.lif')
for filepath in all_files:
    extract_tif(filepath,method = 'max', low_memory=True)
